In [1]:
%load_ext autoreload
%autoreload 2

In [32]:
import models
from sklearn.metrics.pairwise import haversine_distances
import numpy as np
import plotly
import plotly.express as px
import pandas as pd
import geopandas as gpd
import mscthesis

In [55]:
geotweets = mscthesis.read_geotweets('sweden_infered').set_index('userid')

In [56]:
tweetcount = geotweets.groupby('userid').size()
tweetcount

userid
5616                  2459
5976                   356
8981                  1145
32213                  443
392123                 838
                      ... 
764655612525473792      72
778179816889675777      43
782260642770288640      51
790301765744951296      62
794973407964229636    1223
Length: 4056, dtype: int64

In [57]:
geotweets = geotweets.drop(
    labels=tweetcount[tweetcount < 50].index,
)

In [58]:
travel_survey_trips_mean = 3.143905 # PI? It's magic!
travel_survey_trips_std = 1.880373
samples = []
def save_sample(uid, day, timeslot, d, sample):
    s = [uid, day, timeslot]
    if sample[0] == 'region':
        s = s + ['region', sample[1], sample[2], sample[3]]
    else:
        s = s + ['point', -1, sample[1], sample[2]]
    d.append(s)

i = 0
for uid in geotweets.index.unique():
    usertweets = geotweets.loc[uid]
    m = models.SongModel(p=0.66, gamma=0.6, zipf=-1.2)
    m.fit(usertweets)
    home_location = usertweets[usertweets['label'] == 'home']
    for day in range(7*60):
        prev = (
            "region", 
            home_location.iloc[0]['region'], 
            home_location.iloc[0]['latitude'], 
            home_location.iloc[0]['longitude'],
        )
        save_sample(uid, day, 0, samples, prev)
        num_of_trips = max(1, int(round(np.random.normal(travel_survey_trips_mean, travel_survey_trips_std))))
        for timeslot in range(num_of_trips):
            prev = m.sample(prev)
            save_sample(uid, day, timeslot+1, samples, prev)
    i += 1
    if i % 250 == 0:
        print("done with", i)

done with 250
done with 500
done with 750
done with 1000
done with 1250
done with 1500
done with 1750
done with 2000
done with 2250
done with 2500
done with 2750
done with 3000
done with 3250
done with 3500


In [59]:
samplesx = pd.DataFrame(
    samples,
    columns=['userid', 'day', 'timeslot', 'kind', 'region', 'latitude', 'longitude'],
).set_index('userid')

In [60]:
samplesx.loc[5616]

,day,timeslot,kind,region,latitude,longitude
userid,,,,,,
5616,0,0,region,5,59.426889,17.954336
5616,0,1,region,5,59.426889,17.954336
5616,0,2,region,10,59.402809,17.944212
5616,0,3,region,11,59.455540,17.956317
5616,0,4,region,344,59.342576,18.048046
...,...,...,...,...,...,...
5616,418,2,region,5,59.426889,17.954336
5616,419,0,region,5,59.426889,17.954336
5616,419,1,region,220,57.736323,18.879987


In [61]:
samplesx.to_csv('./../../dbs/sweden/visits-song-infered-timeslots-60w.csv')